In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

reading the files using pandas

In [ ]:
train_dt = pd.read_csv('../input/mobile-price-classification/train.csv')
test_dt = pd.read_csv('../input/mobile-price-classification/test.csv')

displaying the read data to check if its imported properly

In [ ]:
train_dt.head()

In [ ]:
test_dt.head()

In [ ]:
train_dt.describe()

In [ ]:
train_dt.info()

checking null values in the dataset

In [ ]:
print(train_dt.isnull().sum())

countplot to see number of cores

In [ ]:
sns.countplot(data=train_dt, x='n_cores')

countplot to see number of internal memory

In [ ]:
plt.subplots(figsize=(20,8))
sns.countplot(data=train_dt, x='int_memory')

comparison to see number of sim's prominent in phone availabel in the market

In [ ]:
sns.countplot(data=train_dt, x='dual_sim')

heatmap to know the relation between various parameters. But in the plot we can see number of cores if related to price.

In [ ]:
plt.subplots(figsize=(20,8))
sns.heatmap(data=train_dt.corr(), annot=True, cbar=True)

In [ ]:
sns.barplot(data=train_dt, x='price_range', y='ram')

moving the target to variable to y

In [ ]:
x= train_dt.drop('price_range', axis=1)
y = train_dt['price_range']

splitting data 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

In [ ]:
lr.predict(x_test)

In [ ]:
lr.score(x_test, y_test)

In [ ]:
confusion_matrix(y_test, lr.predict(x_test))

In [ ]:
rfc = RandomForestClassifier(n_estimators=500, max_depth=11)

In [ ]:
rfc.fit(x_train, y_train)

In [ ]:
rfc_pred = rfc.predict(x_test)

In [ ]:
accuracy_score(y_test, rfc_pred)

In [ ]:
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(x_train, y_train)

In [ ]:
knn_best=knn_gs.best_estimator_
knn_best

In [ ]:
params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rfc, params_rf, cv=5)
rf_gs.fit(x_train, y_train)

In [ ]:
rf_best=rf_gs.best_estimator_
rf_best

In [ ]:
print('knn score ',knn_best.score(x_test, y_test))
print('random forest score ',rf_best.score(x_test, y_test))
print('logistic score ',lr.score(x_test, y_test))

In [ ]:
estimators=[('knn',knn_best), ('rfc', rf_best), ('lr', lr)]

In [ ]:
ensemble=VotingClassifier(estimators, voting='hard')
ensemble.fit(x_train, y_train)
ensemble.score(x_test, y_test)

In [ ]:
model = pd.Series({'knn':0.925, 'random forest':0.87, 'logistic':0.7, 'ensemble':0.8983})
tab = pd.DataFrame({'score':model})
tab

summary